In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timezone
import time
import os

In [2]:
def connect_mt5(portable=True):
    """Connect to MT5 with portable mode option"""
    
    if portable:
        mt5_path = r"C:\ProgramData\Microsoft\Windows\Start Menu\Programs\MetaTrader 5\terminal64.exe"
        
        # Alternative paths - uncomment if needed:
        # mt5_path = r"C:\Program Files (x86)\MetaTrader 5\terminal64.exe"
        # mt5_path = os.getenv('MT5_PATH', mt5_path)
        
        if not os.path.exists(mt5_path):
            print(f"Warning: MT5 not found at {mt5_path}, trying default initialization")
            portable = False
    
    if portable:
        if not mt5.initialize(
            path=mt5_path,
            login=100985424,
            password="@xX7NkAt",
            server="MetaQuotes-Demo",
            timeout=60000,
            portable=True
        ):
            error = mt5.last_error()
            raise RuntimeError(f"MT5 initialize() failed: {error}")
    else:
        if not mt5.initialize(
            login=100985424,
            password="@xX7NkAt",
            server="MetaQuotes-Demo"
        ):
            error = mt5.last_error()
            raise RuntimeError(f"MT5 initialize() failed: {error}")
    
    account_info = mt5.account_info()
    if account_info:
        print("✓ Connected to MT5")
        print(f"  Account: {account_info.login}")
        print(f"  Balance: ${account_info.balance}")
    else:
        print("✓ Connected to MT5")

In [3]:
def shutdown_mt5():
    mt5.shutdown()
    print("MT5 disconnected")

In [11]:
def get_candle_data(symbol="EURUSD", timeframe=mt5.TIMEFRAME_M5, n=100):
    """Get candle data for analysis"""
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None or len(rates) == 0:
        return None
    
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df
connect_mt5()
df = get_candle_data()
print(df.head(5).T)

✓ Connected to MT5
  Account: 100985424
  Balance: $99985.0
                               0                    1                    2  \
time         2026-01-07 05:35:00  2026-01-07 05:40:00  2026-01-07 05:45:00   
open                     1.16992              1.16996              1.16981   
high                     1.16997              1.16997              1.16991   
low                      1.16986               1.1698              1.16977   
close                    1.16996              1.16981              1.16985   
tick_volume                   79                   78                   71   
spread                         1                    2                    0   
real_volume                    0                    0                    0   

                               3                    4  
time         2026-01-07 05:50:00  2026-01-07 05:55:00  
open                     1.16985              1.16991  
high                     1.16994              1.16994  
low          

In [7]:
def calculate_ema(df, period):
    """Calculate Exponential Moving Average"""
    return df['close'].ewm(span=period, adjust=False).mean()

In [19]:
def detect_ema_crossover(df):
    """
    Detect EMA crossover signals
    Returns: 'bullish', 'bearish', or 'neutral'
    """
    if len(df) < 52:  # Need at least 52 candles for 51 EMA
        return "neutral", None, None
    
    # Calculate EMAs
    df['ema_21'] = calculate_ema(df, 21)
    df['ema_51'] = calculate_ema(df, 51)
    
    # Get last 3 candles for crossover detection
    curr = df.iloc[-1]
    prev = df.iloc[-2]
    prev2 = df.iloc[-3]
    
    # Check for bullish crossover (21 EMA crosses above 51 EMA)
    if (prev['ema_21'] <= prev['ema_51'] and 
        curr['ema_21'] > curr['ema_51']):
        return "bullish", curr, df
    
    # Check for bearish crossover (21 EMA crosses below 51 EMA)
    if (prev['ema_21'] >= prev['ema_51'] and 
        curr['ema_21'] < curr['ema_51']):
        return "bearish", curr, df
    
    return "neutral", curr, df


In [20]:
def place_ema_trade(symbol, signal, curr_candle, df, lot=0.1):
    """
    Place trade based on EMA crossover
    SL: Current candle's high (bearish) or low (bullish)
    TP: 2x SL distance
    """
    if signal not in ["bullish", "bearish"]:
        print("No trade signal (neutral) — skipping trade.")
        return

    # Get symbol info
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Failed to get symbol info for {symbol}")
        return
    
    if not symbol_info.visible:
        if not mt5.symbol_select(symbol, True):
            print(f"Failed to add {symbol} to Market Watch")
            return

    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get tick for {symbol}")
        return
    
    # Entry price
    price = tick.ask if signal == "bullish" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if signal == "bullish" else mt5.ORDER_TYPE_SELL

    # Calculate SL based on current candle
    if signal == "bullish":
        sl_price = curr_candle['low']
        sl_dist = abs(price - sl_price)
    else:
        sl_price = curr_candle['high']
        sl_dist = abs(price - sl_price)
    
    # Set MINIMUM stop distances
    point = symbol_info.point
    
    if symbol in ['USDCNH', 'USDZAR', 'USDTRY']:
        min_pips = 30
    else:
        min_pips = 15
    
    min_distance = min_pips * point * 10
    
    print(f"Symbol: {symbol}, Point: {point}")
    print(f"Calculated SL dist: {sl_dist:.5f} (from candle {'low' if signal == 'bullish' else 'high'})")
    print(f"Minimum required: {min_distance:.5f} ({min_pips} pips)")
    
    # Use the larger of calculated or minimum
    if sl_dist < min_distance:
        print("⚠ SL too tight, using minimum distance")
        sl_dist = min_distance
    
    # TP is 2x SL distance
    tp_dist = 2 * sl_dist
    
    # Calculate final SL/TP
    if signal == "bullish":
        sl = round(price - sl_dist, symbol_info.digits)
        tp = round(price + tp_dist, symbol_info.digits)
    else:
        sl = round(price + sl_dist, symbol_info.digits)
        tp = round(price - tp_dist, symbol_info.digits)
    
    # Validate stops
    if signal == "bullish":
        if sl >= price or tp <= price:
            print("ERROR: Invalid stop levels for bullish trade")
            return
    else:
        if sl <= price or tp >= price:
            print("ERROR: Invalid stop levels for bearish trade")
            return
    
    print(f"Entry: {price} | SL: {sl} | TP: {tp}")
    print(f"Risk/Reward: 1:{tp_dist/sl_dist:.1f}")

    # Try different filling modes
    filling_modes = [mt5.ORDER_FILLING_RETURN, mt5.ORDER_FILLING_IOC, mt5.ORDER_FILLING_FOK]
    
    for filling_type in filling_modes:
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": order_type,
            "price": price,
            "sl": float(sl),
            "tp": float(tp),
            "deviation": 20,
            "magic": 234001,  # Different magic number for EMA strategy
            "comment": f"EMA {signal}",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": filling_type,
        }

        result = mt5.order_send(request)

        if result.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✓ {signal.upper()} trade placed successfully!")
            print(f"  21 EMA: {curr_candle['ema_21']:.5f}")
            print(f"  51 EMA: {curr_candle['ema_51']:.5f}")
            return
        elif result.retcode == 10018:
            print(f"✗ Market is closed for {symbol}")
            return
        elif result.retcode != 10030:
            print(f"✗ Order failed: {result.retcode} - {result.comment}")
            return
    
    print(f"✗ Order failed with all filling modes. Last: {result.retcode} - {result.comment}")

In [21]:
def is_market_open(symbol: str) -> bool:
    """Check if market is open and tradable"""
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        return False

    if symbol_info.trade_mode != mt5.SYMBOL_TRADE_MODE_FULL:
        return False

    if not symbol_info.visible:
        if not mt5.symbol_select(symbol, True):
            return False

    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return False

    if tick.bid <= 0 or tick.ask <= 0:
        return False

    # Check tick freshness
    tick_time = datetime.fromtimestamp(tick.time, tz=timezone.utc)
    now = datetime.now(timezone.utc)

    if (now - tick_time).total_seconds() > 60:
        return False

    # Avoid rollover period
    if (tick_time.hour == 23 and tick_time.minute >= 55) or \
       (tick_time.hour == 0 and tick_time.minute <= 5):
        return False

    return True

In [22]:
def check_existing_position(symbol):
    """Check if there's already an open position for this symbol"""
    positions = mt5.positions_get(symbol=symbol)
    if positions is None:
        return False
    return len(positions) > 0

In [23]:
def run_ema_strategy(symbol, timeframe=mt5.TIMEFRAME_M5):
    """Run the EMA crossover strategy"""
    
    # Check for existing position
    if check_existing_position(symbol):
        print(f"[{symbol}] Already have open position - skipping")
        return
    
    # Get data
    df = get_candle_data(symbol, timeframe, n=100)
    if df is None or len(df) < 52:
        print(f"[{symbol}] Insufficient data")
        return
    
    # Detect signal
    signal, curr_candle, df_with_emas = detect_ema_crossover(df)
    
    if signal == "neutral":
        print(f"[{symbol}] No crossover signal")
        return
    
    print(f"[{symbol}] {signal.upper()} CROSSOVER DETECTED!")
    print(f"  21 EMA: {curr_candle['ema_21']:.5f}")
    print(f"  51 EMA: {curr_candle['ema_51']:.5f}")
    print(f"  Candle: O={curr_candle['open']:.5f}, H={curr_candle['high']:.5f}, "
          f"L={curr_candle['low']:.5f}, C={curr_candle['close']:.5f}")
    
    # Place trade
    place_ema_trade(symbol, signal, curr_candle, df_with_emas)

In [ ]:
# ===================== MAIN EXECUTION =====================

currency_pair_list = ["EURUSD", "GBPUSD", "USDJPY", "AUDUSD", "USDCHF", "USDCAD"]

print("=" * 60)
print("EMA CROSSOVER STRATEGY (21/51 EMA on 5-Min Chart)")
print("=" * 60)

connect_mt5(portable=True)  # Use portable=False if path issues

try:
    last_check = {}  # Track last candle time to avoid duplicate signals
    
    while True:
        current_time = datetime.now()
        print(f"\n[{current_time.strftime('%Y-%m-%d %H:%M:%S')}] Scanning for signals...")
        
        for pair in currency_pair_list:
            try:
                if not is_market_open(pair):
                    print(f"[{pair}] Market not open - skipping")
                    continue
                
                # Get current candle time
                df = get_candle_data(pair, mt5.TIMEFRAME_M5, n=2)
                if df is not None:
                    current_candle_time = df.iloc[-1]['time']
                    
                    # Only check if new candle formed
                    if pair not in last_check or last_check[pair] != current_candle_time:
                        run_ema_strategy(pair, mt5.TIMEFRAME_M5)
                        last_check[pair] = current_candle_time
                
            except Exception as e:
                print(f"[{pair}] Error: {e}")
                continue
        
        # Wait for next check (check every 30 seconds)
        print("\n" + "-" * 60)
        time.sleep(30)

except KeyboardInterrupt:
    print("\n\nStopping strategy...")

finally:
    shutdown_mt5()

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timezone
import time
import os

def connect_mt5(portable=True):
    """Connect to MT5 with portable mode option"""
    
    if portable:
        mt5_path = r"C:\ProgramData\Microsoft\Windows\Start Menu\Programs\MetaTrader 5\terminal64.exe"
        
        # Alternative paths - uncomment if needed:
        # mt5_path = r"C:\Program Files (x86)\MetaTrader 5\terminal64.exe"
        # mt5_path = os.getenv('MT5_PATH', mt5_path)
        
        if not os.path.exists(mt5_path):
            print(f"Warning: MT5 not found at {mt5_path}, trying default initialization")
            portable = False
    
    if portable:
        if not mt5.initialize(
            path=mt5_path,
            login=100985424,
            password="@xX7NkAt",
            server="MetaQuotes-Demo",
            timeout=60000,
            portable=True
        ):
            error = mt5.last_error()
            raise RuntimeError(f"MT5 initialize() failed: {error}")
    else:
        if not mt5.initialize(
            login=100985424,
            password="@xX7NkAt",
            server="MetaQuotes-Demo"
        ):
            error = mt5.last_error()
            raise RuntimeError(f"MT5 initialize() failed: {error}")
    
    account_info = mt5.account_info()
    if account_info:
        print("✓ Connected to MT5")
        print(f"  Account: {account_info.login}")
        print(f"  Balance: ${account_info.balance}")
    else:
        print("✓ Connected to MT5")


def shutdown_mt5():
    mt5.shutdown()
    print("MT5 disconnected")


def get_candle_data(symbol="EURUSD", timeframe=mt5.TIMEFRAME_M5, n=100):
    """Get candle data for analysis"""
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None or len(rates) == 0:
        return None
    
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df


def calculate_ema(df, period):
    """Calculate Exponential Moving Average"""
    return df['close'].ewm(span=period, adjust=False).mean()


def calculate_rsi(df, period=14):
    """Calculate Relative Strength Index (RSI)"""
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


def detect_ema_crossover(df, rsi_threshold_bullish=60, rsi_threshold_bearish=40):
    """
    Detect EMA crossover signals WITH RSI filter
    - Bullish: 21 EMA crosses above 51 EMA AND RSI >= 60
    - Bearish: 21 EMA crosses below 51 EMA AND RSI <= 40
    Returns: 'bullish', 'bearish', or 'neutral'
    """
    if len(df) < 52:  # Need at least 52 candles for 51 EMA
        return "neutral", None, None
    
    # Calculate EMAs
    df['ema_21'] = calculate_ema(df, 21)
    df['ema_51'] = calculate_ema(df, 51)
    
    # Calculate RSI
    df['rsi'] = calculate_rsi(df, period=14)
    
    # Get last 3 candles for crossover detection
    curr = df.iloc[-1]
    prev = df.iloc[-2]
    prev2 = df.iloc[-3]
    
    # Check for bullish crossover (21 EMA crosses above 51 EMA) + RSI >= 60
    if (prev['ema_21'] <= prev['ema_51'] and 
        curr['ema_21'] > curr['ema_51'] and
        curr['rsi'] >= rsi_threshold_bullish):
        return "bullish", curr, df
    
    # Check for bearish crossover (21 EMA crosses below 51 EMA) + RSI <= 40
    if (prev['ema_21'] >= prev['ema_51'] and 
        curr['ema_21'] < curr['ema_51'] and
        curr['rsi'] <= rsi_threshold_bearish):
        return "bearish", curr, df
    
    # Check if crossover occurred but RSI filter not met
    if prev['ema_21'] <= prev['ema_51'] and curr['ema_21'] > curr['ema_51']:
        print(f"  ℹ Bullish crossover detected but RSI {curr['rsi']:.2f} < {rsi_threshold_bullish} (rejected)")
    
    if prev['ema_21'] >= prev['ema_51'] and curr['ema_21'] < curr['ema_51']:
        print(f"  ℹ Bearish crossover detected but RSI {curr['rsi']:.2f} > {rsi_threshold_bearish} (rejected)")
    
    return "neutral", curr, df


def place_ema_trade(symbol, signal, curr_candle, df, lot=0.1):
    """
    Place trade based on EMA crossover + RSI filter
    SL: Current candle's high (bearish) or low (bullish)
    TP: 2x SL distance
    """
    if signal not in ["bullish", "bearish"]:
        print("No trade signal (neutral) — skipping trade.")
        return

    # Get symbol info
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Failed to get symbol info for {symbol}")
        return
    
    if not symbol_info.visible:
        if not mt5.symbol_select(symbol, True):
            print(f"Failed to add {symbol} to Market Watch")
            return

    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get tick for {symbol}")
        return
    
    # Entry price
    price = tick.ask if signal == "bullish" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if signal == "bullish" else mt5.ORDER_TYPE_SELL

    # Calculate SL based on current candle
    if signal == "bullish":
        sl_price = curr_candle['low']
        sl_dist = abs(price - sl_price)
    else:
        sl_price = curr_candle['high']
        sl_dist = abs(price - sl_price)
    
    # Set MINIMUM stop distances
    point = symbol_info.point
    
    if symbol in ['USDCNH', 'USDZAR', 'USDTRY']:
        min_pips = 30
    else:
        min_pips = 15
    
    min_distance = min_pips * point * 10
    
    print(f"Symbol: {symbol}, Point: {point}")
    print(f"Calculated SL dist: {sl_dist:.5f} (from candle {'low' if signal == 'bullish' else 'high'})")
    print(f"Minimum required: {min_distance:.5f} ({min_pips} pips)")
    
    # Use the larger of calculated or minimum
    if sl_dist < min_distance:
        print("⚠ SL too tight, using minimum distance")
        sl_dist = min_distance
    
    # TP is 2x SL distance
    tp_dist = 2 * sl_dist
    
    # Calculate final SL/TP
    if signal == "bullish":
        sl = round(price - sl_dist, symbol_info.digits)
        tp = round(price + tp_dist, symbol_info.digits)
    else:
        sl = round(price + sl_dist, symbol_info.digits)
        tp = round(price - tp_dist, symbol_info.digits)
    
    # Validate stops
    if signal == "bullish":
        if sl >= price or tp <= price:
            print("ERROR: Invalid stop levels for bullish trade")
            return
    else:
        if sl <= price or tp >= price:
            print("ERROR: Invalid stop levels for bearish trade")
            return
    
    print(f"Entry: {price} | SL: {sl} | TP: {tp}")
    print(f"Risk/Reward: 1:{tp_dist/sl_dist:.1f}")

    # Try different filling modes
    filling_modes = [mt5.ORDER_FILLING_RETURN, mt5.ORDER_FILLING_IOC, mt5.ORDER_FILLING_FOK]
    
    for filling_type in filling_modes:
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": order_type,
            "price": price,
            "sl": float(sl),
            "tp": float(tp),
            "deviation": 20,
            "magic": 234001,  # Different magic number for EMA strategy
            "comment": f"EMA+RSI {signal}",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": filling_type,
        }

        result = mt5.order_send(request)

        if result.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✓ {signal.upper()} trade placed successfully!")
            print(f"  21 EMA: {curr_candle['ema_21']:.5f}")
            print(f"  51 EMA: {curr_candle['ema_51']:.5f}")
            print(f"  RSI: {curr_candle['rsi']:.2f}")
            return
        elif result.retcode == 10018:
            print(f"✗ Market is closed for {symbol}")
            return
        elif result.retcode != 10030:
            print(f"✗ Order failed: {result.retcode} - {result.comment}")
            return
    
    print(f"✗ Order failed with all filling modes. Last: {result.retcode} - {result.comment}")


def is_market_open(symbol: str) -> bool:
    """Check if market is open and tradable"""
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        return False

    if symbol_info.trade_mode != mt5.SYMBOL_TRADE_MODE_FULL:
        return False

    if not symbol_info.visible:
        if not mt5.symbol_select(symbol, True):
            return False

    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return False

    if tick.bid <= 0 or tick.ask <= 0:
        return False

    # Check tick freshness
    tick_time = datetime.fromtimestamp(tick.time, tz=timezone.utc)
    now = datetime.now(timezone.utc)

    if (now - tick_time).total_seconds() > 60:
        return False

    # Avoid rollover period
    if (tick_time.hour == 23 and tick_time.minute >= 55) or \
       (tick_time.hour == 0 and tick_time.minute <= 5):
        return False

    return True


def check_existing_position(symbol):
    """Check if there's already an open position for this symbol"""
    positions = mt5.positions_get(symbol=symbol)
    if positions is None:
        return False
    return len(positions) > 0


def run_ema_strategy(symbol, timeframe=mt5.TIMEFRAME_M5):
    """Run the EMA crossover strategy with RSI filter"""
    
    # Check for existing position
    if check_existing_position(symbol):
        print(f"[{symbol}] Already have open position - skipping")
        return
    
    # Get data
    df = get_candle_data(symbol, timeframe, n=100)
    if df is None or len(df) < 52:
        print(f"[{symbol}] Insufficient data")
        return
    
    # Detect signal (with RSI filter)
    signal, curr_candle, df_with_emas = detect_ema_crossover(df)
    
    if signal == "neutral":
        print(f"[{symbol}] No valid signal")
        return
    
    print(f"[{symbol}] {signal.upper()} CROSSOVER DETECTED (RSI CONFIRMED)!")
    print(f"  21 EMA: {curr_candle['ema_21']:.5f}")
    print(f"  51 EMA: {curr_candle['ema_51']:.5f}")
    print(f"  RSI: {curr_candle['rsi']:.2f}")
    print(f"  Candle: O={curr_candle['open']:.5f}, H={curr_candle['high']:.5f}, "
          f"L={curr_candle['low']:.5f}, C={curr_candle['close']:.5f}")
    
    # Place trade
    place_ema_trade(symbol, signal, curr_candle, df_with_emas)


# ===================== MAIN EXECUTION =====================

currency_pair_list = ["EURUSD", "GBPUSD", "USDJPY", "AUDUSD", "USDCHF", "USDCAD"]

print("=" * 60)
print("EMA CROSSOVER + RSI FILTER STRATEGY")
print("21/51 EMA on 5-Min Chart + RSI(14)")
print("Bullish: RSI >= 60 | Bearish: RSI <= 40")
print("=" * 60)

connect_mt5(portable=True)  # Use portable=False if path issues

try:
    last_check = {}  # Track last candle time to avoid duplicate signals
    
    while True:
        current_time = datetime.now()
        print(f"\n[{current_time.strftime('%Y-%m-%d %H:%M:%S')}] Scanning for signals...")
        
        for pair in currency_pair_list:
            try:
                if not is_market_open(pair):
                    print(f"[{pair}] Market not open - skipping")
                    continue
                
                # Get current candle time
                df = get_candle_data(pair, mt5.TIMEFRAME_M5, n=2)
                if df is not None:
                    current_candle_time = df.iloc[-1]['time']
                    
                    # Only check if new candle formed
                    if pair not in last_check or last_check[pair] != current_candle_time:
                        run_ema_strategy(pair, mt5.TIMEFRAME_M5)
                        last_check[pair] = current_candle_time
                
            except Exception as e:
                print(f"[{pair}] Error: {e}")
                continue
        
        # Wait for next check (check every 30 seconds)
        print("\n" + "-" * 60)
        time.sleep(30)

except KeyboardInterrupt:
    print("\n\nStopping strategy...")

finally:
    shutdown_mt5()